In [93]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 15, 12


# import seaborn as sns

# from imblearn.over_sampling import SMOTE

In [79]:
df = pd.read_csv("/Users/ianchang/0.Workspaces/slaCoefficientProject/Origination Data/DTT SLA Summary - Overall Summary.csv").drop(columns=["Unnamed: 0", "partner_id", "partner", "xfr_request_id", "request_id", "true_SLA", "runtime_per_day", "payment_code"])

In [80]:
df['submit_date'] = pd.to_datetime(df['submit_date'])
df["submit_date"] = df["submit_date"].dt.floor('D')

# Extract year and month, then create the submit_month column
df['submit_month'] = df['submit_date'].dt.to_period('M')

In [81]:
origination_by_month = df[["on_time", "submit_month"]].groupby(by="submit_month").mean().reset_index()


In [82]:
# fig, ax = plt.subplots(figsize = (10,5))
# plt.style.use('bmh')

# ax.plot(origination_by_month["submit_month"].astype("str"), origination_by_month["on_time"], marker="v", label="Monthly OTD%", color="blue")


# ax.set_ylim(0.9, 1)
# ax.axhline(y=0.95, color='red', linestyle='--', label="Desired Level of OTD%")
# ax.set_title("Monthly Trend of On Time Delivery Percentage (OTD%)")
# ax.set_xlabel("Month")
# ax.set_ylabel("% in Decimals")

# plt.xticks(rotation=45)
# plt.legend()
# plt.show()

In [83]:
import random
import string

def get_random_string(length=8):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

df['author'] = df['author'].str.replace(r'@.*', '', regex=True)

unique_authors = df['author'].unique()

author_mapping = {}

for author in unique_authors:
    author_mapping[author] = get_random_string()

df['author_anonymous'] = df['author'].map(author_mapping)



In [92]:
# df = df.sort_values(by="author_anonymous")

In [85]:
# df.groupby(by=["author_anonymous", "asset_type", "reason_code", "source_language", "target_language"]).agg({"runtime_minutes": "sum"}).head(50)

In [86]:
# Initialize  
df["expected_daily_output"] = 0

for index, row in df.iterrows():
    if row["asset_type"] == "SUBTITLE":
        if row["reason_code"] == "STANDARD":
            df.at[index, "expected_daily_output"] = 15
        elif row["reason_code"] == "RUSH":
            df.at[index, "expected_daily_output"] = 30

    elif row["asset_type"] == "CLOSED_CAPTIONING":
        if row["original_audio_language"] == row["target_language"]:
            if row["reason_code"] == "STANDARD":
                df.at[index, "expected_daily_output"] = 30
            elif row["reason_code"] == "RUSH":
                df.at[index, "expected_daily_output"] = 60
        else:
            if row["reason_code"] == "STANDARD":
                df.at[index, "expected_daily_output"] = 15
            elif row["reason_code"] == "RUSH":
                df.at[index, "expected_daily_output"] = 30
    
    elif row["asset_type"] == "LANGUAGE_TEMPLATE":
        if row["reason_code"] == "STANDARD":
            df.at[index, "expected_daily_output"] = 15
        elif row["reason_code"] == "RUSH":
            df.at[index, "expected_daily_output"] = 30

df = df[['author_anonymous','title_id', 'package_id', 'title_desc', 'asset_type',
       'content_category', 'reason_code', 'original_audio_language',
       'source_language', 'target_language',"expected_daily_output", 'runtime_minutes',
        'start_date', 'submit_date', 'due_date', 'on_time',
       'billing_events']]

In [87]:
import pandas as pd

def calculate_business_days(start_date_str, end_date_str):
    """
    Calculate the number of business days between two dates excluding weekends.

    Parameters:
    start_date_str (str): The start date in YYYY-MM-DD format.
    end_date_str (str): The end date in YYYY-MM-DD format.

    Returns:
    int: The number of business days between the two dates.
    """
    # Convert the date strings to datetime objects
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)
    
    # Calculate the number of business days (excluding weekends)
    business_days = pd.date_range(start=start_date, end=end_date, freq='B')
    
    # Return the count of business days
    return len(business_days)


In [94]:
for index, row in df.iterrows():
    df.at[index,"num_days_spent"] = calculate_business_days(row["start_date"], row["submit_date"])

df = df[['author_anonymous','title_id', 'package_id', 'asset_type',
       'content_category', 'reason_code', 'original_audio_language',
       'source_language', 'target_language',"expected_daily_output", 'runtime_minutes',
        'start_date', 'submit_date','num_days_spent', 'due_date', 'on_time',
       'billing_events']]

In [101]:
df

,author_anonymous,title_id,package_id,asset_type,content_category,reason_code,original_audio_language,source_language,target_language,expected_daily_output,runtime_minutes,start_date,submit_date,num_days_spent,due_date,on_time,billing_events
4615,AADQWVjI,81387029,2014903,SUBTITLE,VIDEO,STANDARD,en,en,zh-Hant,15,31.3,2022-01-14,2022-01-25,8.0,2022-01-25,1,511.0
4617,AADQWVjI,81387028,2014906,SUBTITLE,VIDEO,STANDARD,en,en,zh-Hant,15,30.1,2022-01-09,2022-01-25,12.0,2022-01-25,1,397.0
4633,AADQWVjI,81012353,1695724,SUBTITLE,VIDEO,STANDARD,es-ES,en,zh-Hant,15,41.3,2022-01-03,2022-01-26,18.0,2022-01-26,1,319.0
9701,AADQWVjI,81317479,2000072,SUBTITLE,VIDEO,STANDARD,fi,en,zh-Hant,15,49.6,2022-07-13,2022-07-25,9.0,2022-07-25,1,430.0
8297,AADQWVjI,81317472,2000059,SUBTITLE,VIDEO,STANDARD,fi,en,zh-Hant,15,43.6,2022-06-11,2022-06-12,0.0,2022-06-14,1,448.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9022,zwAuJIac,81275215,2075649,SUBTITLE,VIDEO,STANDARD,en,en,uk,15,114.0,2022-06-24,2022-07-04,7.0,2022-07-06,1,1641.0
9909,zwAuJIac,81285541,2066210,SUBTITLE,VIDEO,STANDARD,de,en,uk,15,56.0,2022-07-29,2022-07-31,1.0,2022-08-03,1,587.0
5090,zwAuJIac,80010542,1182760,SUBTITLE,VIDEO,STANDARD,en,en,uk,15,62.0,2022-02-20,2022-02-23,3.0,2022-02-23,1,623.0
10632,zwAuJIac,81455917,2067669,SUBTITLE,VIDEO,STANDARD,en,en,uk,15,46.5,2022-10-07,2022-10-11,3.0,2022-10-12,1,408.0
